In [0]:
"Hello world!"

Out[1]: 'Hello world!'

# Data Ingestion and Cleansing
1. Setting up databricks community environment

In [0]:
# Print the configuration of the Spark session. Use spark configuration to update parameters
# Databricks notebooks offer the display() function to print the output of a cell in a user-friendly manner.
# Display all parameters
from pyspark import SparkConf
display(SparkConf().getAll())

# Alternate command using sql('Set -v')
# display(spark.sql("Set -v"))

# Alternate command using sparkContext.getConf().getAll()
# display(spark.sparkContext.getConf().getAll())

_1 _2 spark.executor.memory 8278m spark.ui.port 42407 spark.executor.extraClassPath /databricks/spark/dbconf/log4j/executor:/databricks/spark/dbconf/jets3t/:/databricks/spark/dbconf/hadoop:/databricks/hive/conf:/databricks/jars/----glue-catalog-spark3.1-client--glue-catalog-client-common_deploy.jar:/databricks/jars/----glue-catalog-spark3.1-client--glue-catalog-hive2-client_deploy.jar:/databricks/jars/----glue-catalog-spark3.1-client--glue-catalog-shim-common_deploy.jar:/databricks/jars/----glue-catalog-spark3.1-client--glue-catalog-shim-hive1_deploy.jar:/databricks/jars/----glue-catalog-spark3.1-client--glue-catalog-shim-hive2_deploy.jar:/databricks/jars/----glue-catalog-spark3.1-client--glue-catalog-shim-loader_deploy.jar:/databricks/jars/----jackson_annotations_shaded--libjackson-annotations.jar:/databricks/jars/----jackson_core_shaded--libjackson-core.jar:/databricks/jars/----jackson_databind_shaded--libjackson-databind.jar:/databricks/jars/----jackson_datatype_joda_shaded--libjackson-datatype-joda.jar:/databricks/jars/----scalapb_090--com.lihaoyi__fastparse_2.12__2.1.3_shaded.jar:/databricks/jars/----scalapb_090--com.lihaoyi__sourcecode_2.12__0.1.7_shaded.jar:/databricks/jars/----scalapb_090--runtime-unshaded-jetty9-hadoop1_2.12_deploy_shaded.jar:/databricks/jars/----workspace_spark_3_1--common--kvstore--kvstore-hive-2.3__hadoop-2.7_2.12_deploy.jar:/databricks/jars/----workspace_spark_3_1--common--network-common--network-common-hive-2.3__hadoop-2.7_2.12_deploy.jar:/databricks/jars/----workspace_spark_3_1--common--network-shuffle--network-shuffle-hive-2.3__hadoop-2.7_2.12_deploy.jar:/databricks/jars/----workspace_spark_3_1--common--sketch--sketch-hive-2.3__hadoop-2.7_2.12_deploy.jar:/databricks/jars/----workspace_spark_3_1--common--tags--tags-hive-2.3__hadoop-2.7_2.12_deploy.jar:/databricks/jars/----workspace_spark_3_1--common--unsafe--unsafe-hive-2.3__hadoop-2.7_2.12_deploy.jar:/databricks/jars/----workspace_spark_3_1--core--core-hive-2.3__hadoop-2.7_2.12_deploy.jar:/databricks/jars/----workspace_spark_3_1--core--libcore_generated_resources.jar:/databricks/jars/----workspace_spark_3_1--core--libcore_resources.jar:/databricks/jars/----workspace_spark_3_1--core--proto-hive-2.3__hadoop-2.7_2.12_deploy.jar:/databricks/jars/----workspace_spark_3_1--graphx--graphx-hive-2.3__hadoop-2.7_2.12_deploy.jar:/databricks/jars/----workspace_spark_3_1--launcher--launcher-hive-2.3__hadoop-2.7_2.12_deploy.jar:/databricks/jars/----workspace_spark_3_1--maven-trees--hive-2.3__hadoop-2.7--antlr--antlr--antlr__antlr__2.7.7.jar:/databricks/jars/----workspace_spark_3_1--maven-trees--hive-2.3__hadoop-2.7--com.amazonaws--amazon-kinesis-client--com.amazonaws__amazon-kinesis-client__1.12.0.jar:/databricks/jars/----workspace_spark_3_1--maven-trees--hive-2.3__hadoop-2.7--com.amazonaws--aws-java-sdk-autoscaling--com.amazonaws__aws-java-sdk-autoscaling__1.11.655.jar:/databricks/jars/----workspace_spark_3_1--maven-trees--hive-2.3__hadoop-2.7--com.amazonaws--aws-java-sdk-cloudformation--com.amazonaws__aws-java-sdk-cloudformation__1.11.655.jar:/databricks/jars/----workspace_spark_3_1--maven-trees--hive-2.3__hadoop-2.7--com.amazonaws--aws-java-sdk-cloudfront--com.amazonaws__aws-java-sdk-cloudfront__1.11.655.jar:/databricks/jars/----workspace_spark_3_1--maven-trees--hive-2.3__hadoop-2.7--com.amazonaws--aws-java-sdk-cloudhsm--com.amazonaws__aws-java-sdk-cloudhsm__1.11.655.jar:/databricks/jars/----workspace_spark_3_1--maven-trees--hive-2.3__hadoop-2.7--com.amazonaws--aws-java-sdk-cloudsearch--com.amazonaws__aws-java-sdk-cloudsearch__1.11.655.jar:/databricks/jars/----workspace_spark_3_1--maven-trees--hive-2.3__hadoop-2.7--com.amazonaws--aws-java-sdk-cloudtrail--com.amazonaws__aws-java-sdk-cloudtrail__1.11.655.jar:/databricks/jars/----workspace_spark_3_1--maven-trees--hive-2.3__hadoop-2.7--com.amazonaws--aws-java-sdk-cloudwatch--com.amazonaws__aws-java-sdk-cloudwatch__1.11.655.jar:/databricks/jars/----workspace_spark_3_1--maven-trees--hive-2.3__hadoop-2.7--com.ama

##### Create directories for different layer wise operations on datalake
Use dbutils.fs to perform different operations on the Databricks file system. Create datalake directory under root directory as dbutils.fs('/datalake').
Data lake contains 3 layers that are self explanatory : 1. Raw data layer 2. Curated data layer 3. Serving data layer

In [0]:
dbutils.fs.mkdirs('/datalake')
dbutils.fs.mkdirs('datalake/raw')
dbutils.fs.mkdirs('datalake/curated')
dbutils.fs.mkdirs('datalake/serving')
# List contents of datalake to see the effect
display(dbutils.fs.ls('/datalake'))

path,name,size
dbfs:/datalake/curated/,curated/,0
dbfs:/datalake/raw/,raw/,0
dbfs:/datalake/serving/,serving/,0


Move the data into the data lake’s raw layer using Databricks Utilities. The raw layer is the cornerstone of the data lake because as long as the data is available in it then we can reconstruct both the curated and serving layers via a backfill process, whereas if we lose the data from the raw layer then we’d have to fetch it again from external systems which isn’t always possible.


Get the data once using shell script below (%sh instantiates shell, alternatively use Data Bricks web terminal). This calls 'world development indicators dataset (provided by world bank)' to the local file system of cluster's driver node.

In [0]:
%sh
cd /
mkdir wb_input_data
cd wb_input_data
DATA_URL='https://databank.worldbank.org/data/download/WDI_csv.zip'
wget -O world_development_indicators.zip $DATA_URL
unzip world_development_indicators.zip
rm world_development_indicators.zip

--2022-02-05 14:01:55--  https://databank.worldbank.org/data/download/WDI_csv.zip
Resolving databank.worldbank.org (databank.worldbank.org)... 192.86.102.146
Connecting to databank.worldbank.org (databank.worldbank.org)|192.86.102.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67669068 (65M) [application/x-zip-compressed]
Saving to: ‘world_development_indicators.zip’

     0K .......... .......... .......... .......... ..........  0%  166K 6m38s
    50K .......... .......... .......... .......... ..........  0%  332K 4m58s
   100K .......... .......... .......... .......... ..........  0%  662K 3m52s
   150K .......... .......... .......... .......... ..........  0%  333K 3m43s
   200K .......... .......... .......... .......... ..........  0%  665K 3m18s
   250K .......... .......... .......... .......... ..........  0%  660K 3m2s
   300K .......... .......... .......... .......... ..........  0%  667K 2m50s
   350K .......... .......... .......... .....

In [0]:
%sh
ls /wb_input_data/

WDICountry-Series.csv
WDICountry.csv
WDIData.csv
WDIFootNote.csv
WDISeries-Time.csv
WDISeries.csv


In [0]:
# Copy downloaded data from cluster driver node directory to datalake directory's raw layer. Date parameter is for reviewing logs later in time.
from datetime import datetime
current_date = datetime.today().strftime('%Y%m%d')
dbutils.fs.cp('file:/wb_input_data', f'dbfs:/datalake/raw/world_development_indicators/date={current_date}/', recurse=True)

Out[10]: True

Below is a similar procedure for acquiring CO2 emmissions data. 
CO2 emissions from passenger cars (from 2017 to 2020) - Provided by the European Environment Agency (in JSON format): This dataset will be used for an operational use-case to build a dashboard to monitor multiple metrics extracted from the data.

In [0]:
%sh
apt-get install jq
cd /
mkdir eea_input_data
cd eea_input_data/
BASE_URL="https://discodata.eea.europa.eu"
TABLE="CO2Emission.latest.co2cars"
ORDER="ORDER%20BY%20%27Enedc%20(g%2Fkm)%27%20DESC"
for YEAR in 2017 2018 2019
do
    CONDITION="year%20%3D%20$YEAR"
    SQL_QUERY="SELECT%20*%20FROM%20$TABLE%20WHERE%20$CONDITION%20$ORDER"
    FULL_URL="$BASE_URL/sql?query=$SQL_QUERY&p=1&nrOfHits=100000"
    curl $FULL_URL | jq '.results' > co2_emissions_passenger_cars_$YEAR.json
done

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  jq
0 upgraded, 1 newly installed, 0 to remove and 21 not upgraded.
Need to get 50.2 kB of archives.
After this operation, 99.3 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 jq amd64 1.6-1ubuntu0.20.04.1 [50.2 kB]
debconf: delaying package configuration, since apt-utils is not installed
Fetched 50.2 kB in 1s (79.5 kB/s)
Selecting previously unselected package jq.
(Reading database ... 
(Reading database ... 5%
(Reading database ... 10%
(Reading database ... 15%
(Reading database ... 20%
(Reading database ... 25%
(Reading database ... 30%
(Reading database ... 35%
(Reading database ... 40%
(Reading database ... 45%
(Reading database ... 50%
(Reading database ... 55%
(Reading database ... 60%
(Reading database ... 65%
(Reading database ... 70%
(Reading database ... 75%
(Reading database ... 80%
(Re

In [0]:
%sh
ls /eea_input_data

co2_emissions_passenger_cars_2017.json
co2_emissions_passenger_cars_2018.json
co2_emissions_passenger_cars_2019.json


Move files to datalake/raw layer

In [0]:
for year in [2017, 2018, 2019]:
  file_name = f'co2_emissions_passenger_cars_{year}.json'
  dbutils.fs.cp(
    f'file:/eea_input_data/{file_name}', 
    f'dbfs:/datalake/raw/co2_passenger_cars_emissions/year={year}/{file_name}'
  )

In [0]:
dbutils.fs.ls('dbfs:/datalake/raw/co2_passenger_cars_emissions/year=2017/')

Out[25]: [FileInfo(path='dbfs:/datalake/raw/co2_passenger_cars_emissions/year=2017/co2_emissions_passenger_cars_2017.json', name='co2_emissions_passenger_cars_2017.json', size=82048061)]

Set dataframes by reading via 'spark.read.option().json/csv' for respective json & csv formats. Visualize the schema of two dataframes to confirm the ingestion process.

In [0]:
df_co2_emissions                = (spark.read.option("multiline","true").json('dbfs:/datalake/raw/co2_passenger_cars_emissions'))
df_world_development_indicators = (spark.read.option('header', 'true').csv(f'dbfs:/datalake/raw/world_development_indicators/date={current_date}/WDIData.csv'))

# Printing the schemas of the two dataframes
df_co2_emissions.printSchema()
df_world_development_indicators.printSchema()

root
 |-- At1 (mm): long (nullable = true)
 |-- At2 (mm): long (nullable = true)
 |-- Cn: string (nullable = true)
 |-- Cr: string (nullable = true)
 |-- Ct: string (nullable = true)
 |-- De: double (nullable = true)
 |-- E (g/km): string (nullable = true)
 |-- Enedc (g/km): long (nullable = true)
 |-- Er (g/km): string (nullable = true)
 |-- Ernedc (g/km): double (nullable = true)
 |-- Erwltp (g/km): double (nullable = true)
 |-- Ewltp (g/km): long (nullable = true)
 |-- Fm: string (nullable = true)
 |-- Ft: string (nullable = true)
 |-- ID: long (nullable = true)
 |-- It: string (nullable = true)
 |-- MMS: string (nullable = true)
 |-- MS: string (nullable = true)
 |-- Man: string (nullable = true)
 |-- Mh: string (nullable = true)
 |-- Mk: string (nullable = true)
 |-- Mp: string (nullable = true)
 |-- Mt: long (nullable = true)
 |-- Status: string (nullable = true)
 |-- T: string (nullable = true)
 |-- TAN: string (nullable = true)
 |-- VFN: string (nullable = true)
 |-- Va: string

In [0]:
print(f"Number of records  for CO2 emissions DF: {df_co2_emissions.count()}")
print(f"Number of records  for World Development Indicators: {df_world_development_indicators.count()}")

Number of records  for CO2 emissions DF: 300000
Number of records  for World Development Indicators: 383838


Desriptive statistics of both data frames

In [0]:
display(df_co2_emissions.describe())

summary,At1 (mm),At2 (mm),Cn,Cr,Ct,De,E (g/km),Enedc (g/km),Er (g/km),Ernedc (g/km),Erwltp (g/km),Ewltp (g/km),Fm,Ft,ID,It,MMS,MS,Man,Mh,Mk,Mp,Mt,Status,T,TAN,VFN,Va,Ve,Vf,W (mm),Zr,ec (cm3),ep (KW),m (kg),r,version_file,year,z (Wh/km)
count,283774,276234,299985,299092,298300,10010,0,300000,0,7155,2837,109271,299993,300000,300000,275329,299262,299996,300000,300000,299782,300000,109754,300000,299974,299872,285535,299901,299844,16306,286682,0,299975,248671,299435,300000,300000,300000,55
mean,1630.8959418410425,1637.6312691413802,800.1971752921536,null,null,-0.0017751848151849555,null,265.74504333333334,null,1.6373025856043493,0.0190341910468805,307.64959595867157,null,null,5452443.78782,7.285714285714286,8.0,null,null,null,8.0,null,2197.279279115112,null,474.5888325137403,111.0,null,3.1930333817126228E90,322457.9659583742,0.2551821415429903,2786.1052246042655,null,4193.580708392366,340.51696418158934,2059.763237430494,2.3014666666666668,null,2018.0,217.25454545454545
stddev,57.17631487244919,53.349042291929216,948.4930169775346,null,null,0.04090898083977153,null,46.160696807975754,null,0.3011054880652161,0.18558046935955036,36.00536276566944,null,null,3904773.2634796347,3.5326937743674756,0.0,null,null,null,0.0,null,518.6246416569063,null,5659.605153337306,128.17175976009693,null,3.349852913181437E91,683842.391909468,0.435976917951142,320.44178928427743,null,1161.6180014899765,107.21023694801087,352.8568810417213,22.398605689655657,null,0.816497941758763,22.27123887007817
min,501,501,,,,-0.078,null,185,null,1.0,0.0,11,,,307,,,AT,,AA-IVA,,,0,F,,,,,,0,0,null,898,27,242,1,v15,2017,100
max,2895,2250,unknown,m1,m1,2.0,null,671,null,3.0,1.9,535,P,petrol/electric,15499148,e24*2,ZESZUTA,SK,VOLVO CAR CORPORATION,unknown,ZENVO,VW GROUP PC,4612,P,l50,e9*ks07/46*6716*01,RL-ZE1AEM57A6_0000-SJN-1,ZYX10(H),h,1,4762,null,8382,1103,4160,3250,v20,2019,296


In [0]:
display(df_world_development_indicators.describe())

summary,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,_c65
count,383838,383838,383838,383838,37379,42384,44643,44491,45044,47213,47245,48987,48565,50410,69132,76183,79304,78791,80286,83993,86295,90236,89837,90717,95643,97974,99737,99261,99998,101364,102156,103792,102254,103817,126976,134342,139441,139905,142194,149968,149801,152093,153352,158807,184641,178816,184282,184021,189384,200435,200776,206029,203926,205222,218710,211967,216450,210820,217996,213471,213075,209605,203509,176592,95112,0
mean,null,null,null,null,2.601889436129945E11,2.4828122063759998E11,2.5053466227751028E11,2.5940295290880362E11,2.766423655641177E11,2.9951638606100793E11,3.218653465902251E11,3.3895593685142334E11,3.910909711683894E11,4.2319136619845374E11,3.7312627530723694E11,3.794664810608572E11,4.1376623340422284E11,4.681075523127019E11,5.637952465877424E11,5.710912068690099E11,6.314014606744072E11,6.122062654973693E11,5.705539349264624E11,5.536573767445946E11,4.8216372083058154E11,4.7955351937462616E11,5.1148969373986914E11,6.892090441907516E11,7.547817202445956E11,8.650365449760214E11,8.340266574973143E11,7.941959392234526E11,6.171422526627638E11,6.746004681340612E11,7.278392677985278E11,7.016248836195956E11,7.177102896647844E11,7.932374212078691E11,8.9120597031162E11,9.185719220284064E11,8.864766036395985E11,8.996443446492352E11,8.128974449712367E11,7.889471163793E11,7.221903851572957E11,7.775651520144567E11,9.418836821361178E11,9.021111705811654E11,1.0518143459547124E12,1.0764517220470354E12,1.2160269209822869E12,1.4524105133165725E12,1.6533453224031213E12,1.7262459089245405E12,1.9203341726175684E12,2.224521373825939E12,2.352618239745025E12,2.695734676626705E12,2.849368712645566E12,3.03323291175438E12,3.3064967248378906E12,3.5117151676417783E12,4.0291654115149917E12,5.132563019898995E12,1.0504766625335602E13,null
stddev,null,null,null,null,1.0820188012144344E13,1.1157667993202434E13,1.1638422313372844E13,1.224678282271517E13,1.3193827325926012E13,1.4769049720980533E13,1.6274444622063846E13,1.776483721710273E13,2.0416905930604805E13,2.3109121390745918E13,2.197374681761009E13,2.3862873489965293E13,2.679739612245836E13,2.9515604280666816E13,3.3409584836267957E13,3.3336946762361914E13,3.82552712933314E13,3.654038066612623E13,3.179908701702637E13,2.9215313987584996E13,2.382768770812876E13,2.318628584162323E13,2.587980140516502E13,4.0119894057383234E13,5.053236395322682E13,6.749218714968449E13,6.433502256934794E13,5.426649329856858E13,2.846300526550288E13,3.1290500263935605E13,4.0907423387015055E13,3.524328991600886E13,3.691263662730198E13,4.331544973684792E13,5.186425000377728E13,5.401271924082455E13,4.005682510354743E13,3.832581489320965E13,3.4184080886816402E13,3.651281914188921E13,3.988519675703744E13,4.620085966536751E13,3.8050657156240055E13,5.169488105570802E13,4.501457729963205E13,4.931034550462513E13,5.184663504905697E13,5.591196135110962E13,6.185721333519797E13,6.5770764800641914E13,7.644447401114273E13,8.72221625383127E13,9.375407475709742E13,1.0842857064113614E14,1.1861946367842352E14,1.2554846046297825E14,1.3917607956032966E14,1.5062746071020922E14,1.7544184994791428E14,2.2006302247498756E14,3.7783559183342E14,null
min,Afghanistan,ABW,ARI treatment (% of children under 5 taken to a health provider),AG.AGR.TRAC.NO,-0.00330839030000001,-0.00172840179796196,-0.000155636363545455,-0.000243272726363636,-0.000112838000000001,-0.000641454544454546,-0.000554181818454545,-0.000876727273454545,-0.0002550001,-0.000425453699999996,-0.0002970001,-0.00064314309999999,-0.000224618999999992,-0.000459999996666664,-0.00049591064453125,-0.000168580785369902,-0.000101566299996136,-0.000157833,-0.00013113,-0.000151525860840636,-0.000116415321826935,-0.0

In [0]:
display(df_co2_emissions)

At1 (mm),At2 (mm),Cn,Cr,Ct,De,E (g/km),Enedc (g/km),Er (g/km),Ernedc (g/km),Erwltp (g/km),Ewltp (g/km),Fm,Ft,ID,It,MMS,MS,Man,Mh,Mk,Mp,Mt,Status,T,TAN,VFN,Va,Ve,Vf,W (mm),Zr,ec (cm3),ep (KW),m (kg),r,version_file,year,z (Wh/km)
null,null,BUGATTI BG 744,M1,M1,null,null,575,null,null,null,null,M,petrol,4169823,,BUGATTI F,DE,AA-IVA,AA-IVA,BUGATTI,,null,F,,,,,,null,null,null,7993,1103,1995,1,v20,2019,null
null,null,BUGATTI CHIRON-DIVO,M1,M1,null,null,575,null,null,null,null,M,petrol,4169522,,BUGATTI F,DE,AA-IVA,AA-IVA,BUGATTI,,null,F,,,,,,null,null,null,null,1103,1995,1,v20,2019,null
null,null,BUGATTI CHIRON-DIVO,M1,M1,null,null,575,null,null,null,null,M,petrol,4169522,,BUGATTI F,DE,AA-IVA,AA-IVA,BUGATTI,,null,P,,,,,,null,null,null,null,1103,1995,1,v19,2019,null
null,null,BUGATTI BG 744,M1,M1,null,null,575,null,null,null,null,M,petrol,4169823,,BUGATTI F,DE,AA-IVA,AA-IVA,BUGATTI,,null,P,,,,,,null,null,null,7993,1103,1995,1,v19,2019,null
null,null,AUDI A8,M1,M1,null,null,545,null,null,null,null,M,petrol,4784577,,AUDI,DE,AA-IVA,AA-IVA,AUDI,,null,F,F8,,,,,null,null,null,3993,420,3895,1,v20,2019,null
null,null,AUDI A8,M1,M1,null,null,545,null,null,null,null,M,petrol,4784577,,AUDI,DE,AA-IVA,AA-IVA,AUDI,,null,P,F8,,,,,null,null,null,3993,420,3895,1,v19,2019,null
1747,1670,BUGATTI CHIRON,M1,M1,null,null,516,null,null,null,506,M,petrol,4898529,,BUGATTI F,DE,BUGATTI AUTOMOBILES SAS,BUGATTI,BUGATTI,VW GROUP PC,2150,P,5B,E1*KS07/46*0008*09,,DALAADX1CN,TAD7AD71C002,null,2711,null,7993,1103,2070,1,v19,2019,null
1747,1670,BUGATTI CHIRON,M1,M1,null,null,516,null,null,null,506,M,petrol,4898536,,BUGATTI F,DE,BUGATTI AUTOMOBILES SAS,BUGATTI,BUGATTI,VW GROUP PC,2150,P,5B,E1*KS07/46*0008*10,,DAOAADX1CN,TAD7AD71C002,null,2711,null,7993,110,2070,1,v19,2019,null
1747,1670,BUGATTI CHIRON,M1,M1,null,null,516,null,null,null,505,M,petrol,4898534,,BUGATTI F,DE,BUGATTI AUTOMOBILES SAS,BUGATTI,BUGATTI,VW GROUP PC,2150,P,5B,E1*KS07/46*0008*09,,DALAADX1CN,TAD7AD71C002,null,2711,null,7993,1103,2070,1,v19,2019,null
1747,1670,BUGATTI CHIRON,M1,M1,null,null,516,null,null,null,505,M,petrol,4898519,,BUGATTI F,DE,BUGATTI AUTOMOBILES SAS,BUGATTI,BUGATTI,VW GROUP PC,2150,P,5B,E1*KS07/46*0008*10,,DALAADX1CN,TAD7AD71C002,null,2711,null,7933,1103,2040,1,v19,2019,null


In [0]:
display(df_world_development_indicators)

Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,_c65
Africa Eastern and Southern,AFE,Access to clean fuels and technologies for cooking (% of population),EG.CFT.ACCS.ZS,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,12.205985334256,12.5493332143826,12.8877052744847,13.2263347774934,13.5755915831281,13.9243539706952,14.2660310520972,14.5962971181626,14.9552860566119,15.2810233296453,15.6312911024101,15.9812561302977,16.320474865037,16.6432428558172,16.9946949543723,17.3131682773135,17.6049500630141,null,null,null,null,null
Africa Eastern and Southern,AFE,Access to electricity (% of population),EG.ELC.ACCS.ZS,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,20.0860066130196,23.1031784776438,24.2234301519858,25.0231841252266,26.1763357723345,23.621498439778,27.2994330458403,28.737952342644,28.813735402838,27.4830612476805,28.2333728874701,28.3818359876251,32.2240274434276,32.0464780952398,31.3235788003652,33.3121631315881,38.3804331871331,39.754201347135,42.1682410699846,43.6406611120552,null,null
Africa Eastern and Southern,AFE,"Access to electricity, rural (% of rural population)",EG.ELC.ACCS.RU.ZS,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,8.49160997191625,13.2018746955682,15.2093473536694,13.8135351072229,15.4236146358427,10.9588403149893,16.7624691704676,14.417101517166,16.9779753635741,18.1733410522112,16.2410638636384,15.2959503612632,20.5253527781327,19.4613833438545,17.790698468584,16.5534701416911,23.9078966279348,24.6247249863346,26.8138996635713,28.8411502327975,null,null
Africa Eastern and Southern,AFE,"Access to electricity, urban (% of urban population)",EG.ELC.ACCS.UR.ZS,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,56.0834516422826,54.7701226470266,56.1023300404056,57.0943101230703,57.836866299283,58.9533652971299,59.5469745890136,59.5989593932235,60.9834004395019,61.2272550575374,62.5208242745159,65.4706154133,66.303599202538,66.4960101543106,65.8289881700667,66.9266915686378,68.7221844313438,71.0854179924979,71.9949328691005,73.5898862435969,null,null
Africa Eastern and Southern,AFE,Account ownership at a financial institution or with a mobile-money-service provider (% of population ages 15+),FX.OWN.TOTL.ZS,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
Africa Eastern and Southern,AFE,"Account ownership at a financial institution or with a mobile-money-service provider, female (% of population ages 15+)",FX.OWN.TOTL.FE.ZS,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
Africa Eastern and Southern,AFE,"Account ownership at a financial institution or with a mobile-money-service provider, 

Other than shining a light on topics like inequality and the impact of certain commodities on our planet, the two datasets present an excellent example of input data in a wide range of enterprise scenarios.

# Generate the data curation layer

    Read the data from the raw layer using Spark.
    Apply different filters and data quality checks on the data.
    Write the data in Parquet format to the curated layer directory on DBFS.
    Create tables on top of the data.

Clean up the data and ingest it into the curated layer. The data validation phase is an extremely important part of the pipeline to make sure to drop all the records that fail the data quality checks, so as to ensure data integrity for the curated layer, which is used to build use-case-specific tables in the serving layer. Build Spark-based data pipelines (some of Spark’s most used functionalities) to perform above opeartions.

Workflow:
    Create database 'wdi_curated' for for the World Development Indicators tables on the curated layer
    Create database 'eea_curated' for the passenger cars CO2 emissions (provided by the European Environment Agency) tables on the curated layer
    Put respective csv files under these databases
    Basic cleaning such as replacing spaces with underscores in columns & drop duplicates
    Move to curated layer, partition data in parquet format and finally create tables for the serving layer to read

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS wdi_curated;
CREATE DATABASE IF NOT EXISTS eea_curated;

In [0]:
from datetime import datetime

# Prepare different parts of the data path
raw_layer_base_path = 'dbfs:/datalake/raw/'
wdi_feed = 'world_development_indicators'
partition = datetime.today().strftime('%Y%m%d')

# Read three data files we're interested in
df_wdi_data = (spark.read.option('header', 'true').option('escape', '"').csv(f'{raw_layer_base_path}/{wdi_feed}/date={partition}/WDIData.csv'))

df_wdi_country = (spark.read.option('header', 'true').option('escape', '"').csv(f'{raw_layer_base_path}/{wdi_feed}/date={partition}/WDICountry.csv'))

df_wdi_series = (spark.read.option('header', 'true').option('escape', '"').csv(f'{raw_layer_base_path}/{wdi_feed}/date={partition}/WDISeries.csv'))

In [0]:
import pyspark.sql.functions as F

dfs_wdi = {'WDI_Data'   : df_wdi_data,'WDI_Country': df_wdi_country,'WDI_Series' : df_wdi_series}

# Replace spaces in column names with underscores
for df_name, df in dfs_wdi.items():
  dfs_wdi[df_name] = df.select([F.col(col).alias(col.replace(' ', '_')) for col in df.columns])
  
# Drop null records
for df_name, df in dfs_wdi.items():
  print(f'Number of records of dataframe {df_name} before dropping nulls: {df.count()}')
  dfs_wdi[df_name] = df.na.drop('all')
  print(f'Number of records of dataframe {df_name} after dropping nulls: {dfs_wdi[df_name].count()}')

# Drop duplicates
for df_name, df in dfs_wdi.items():
  print(f'Number of records of dataframe {df_name} before dropping duplicates: {df.count()}')
  dfs_wdi[df_name] = df.distinct()
  print(f'Number of records of dataframe {df_name} after dropping duplicates: {dfs_wdi[df_name].count()}')

Number of records of dataframe WDI_Data before dropping nulls: 383838
Number of records of dataframe WDI_Data after dropping nulls: 383838
Number of records of dataframe WDI_Country before dropping nulls: 270
Number of records of dataframe WDI_Country after dropping nulls: 270
Number of records of dataframe WDI_Series before dropping nulls: 4279
Number of records of dataframe WDI_Series after dropping nulls: 4279
Number of records of dataframe WDI_Data before dropping duplicates: 383838
Number of records of dataframe WDI_Data after dropping duplicates: 383838
Number of records of dataframe WDI_Country before dropping duplicates: 270
Number of records of dataframe WDI_Country after dropping duplicates: 270
Number of records of dataframe WDI_Series before dropping duplicates: 4279
Number of records of dataframe WDI_Series after dropping duplicates: 2329


In [0]:
df_wdi_data    = dfs_wdi['WDI_Data'].where('length(Country_Code) = 3')
df_wdi_country = dfs_wdi['WDI_Country'].where('length(Country_Code) = 3')
df_wdi_series  = dfs_wdi['WDI_Series'].where(~F.col('Series_Code').contains(' '))

Save wdi_data in a table

In [0]:
dbfs_base_path   = 'dbfs:/datalake/curated/wdi'
output_partition = f'year={partition[:4]}/month={partition[4:6]}/day={partition[6:]}/'

# We use coalesce() to reduce the number of partitions before writing the data
# We use saveAsTable() to create tables on top of the data
(
  df_wdi_data
  .coalesce(1)
  .write
  .format('parquet')
  .mode('overwrite')
  .option('path', f'{dbfs_base_path}/data/{output_partition}')
  .saveAsTable('wdi_curated.data')
)
(
  df_wdi_country
  .coalesce(1)
  .write
  .format('parquet')
  .mode('overwrite')
  .option('path', f'{dbfs_base_path}/country/{output_partition}')
  .saveAsTable('wdi_curated.country')
)
(
  df_wdi_series
  .coalesce(1)
  .write
  .format('parquet')
  .mode('overwrite')
  .option('path', f'{dbfs_base_path}/series/{output_partition}')
  .saveAsTable('wdi_curated.series')
)

In [0]:
from datetime import datetime

# Prepare different parts of the data path
raw_layer_base_path = 'dbfs:/datalake/raw'
co2_emissions_feed  = 'co2_passenger_cars_emissions'

# Read the data using Spark
df_co2_emissions = (spark.read.option('multiline','true').json(f'{raw_layer_base_path}/{co2_emissions_feed}'))

In [0]:
import re
import pyspark.sql.functions as F

# Replace spaces in column names with underscores and remove()
df_co2_emissions = (df_co2_emissions.select([F.col(col).alias(re.sub('[()]', '', col.replace(' ', '_'))) for col in df_co2_emissions.columns]))

# Drop null records
print(f'Number of records of CO2 emissions dataframe before dropping nulls: {df_co2_emissions.count()}')
df_co2_emissions = df_co2_emissions.na.drop('all')
print(f'Number of records of CO2 emissions dataframe after dropping nulls: {df_co2_emissions.count()}')

# Drop duplicates
print(f'Number of records of CO2 emissions dataframe before dropping duplicates: {df_co2_emissions.count()}')
df_co2_emissions = df_co2_emissions.distinct()
print(f'Number of records of CO2 emissions dataframe after dropping duplicates: {df_co2_emissions.count()}')

# Filter records with corrupt Member State code - keep values with two uppercase letters
print(f'Number of records of CO2 emissions datarame before MS filter: {df_co2_emissions.count()}')
df_co2_emissions = df_co2_emissions.filter(df_co2_emissions['MS'].rlike('^[A-Z][A-Z]$'))
print(f'Number of records of CO2 emissions dataframe adter MS filter: {df_co2_emissions.count()}')

Number of records of CO2 emissions dataframe before dropping nulls: 300000
Number of records of CO2 emissions dataframe after dropping nulls: 300000
Number of records of CO2 emissions dataframe before dropping duplicates: 300000
Number of records of CO2 emissions dataframe after dropping duplicates: 300000
Number of records of CO2 emissions datarame before MS filter: 300000
Number of records of CO2 emissions dataframe adter MS filter: 299996


In [0]:
dbfs_base_path = 'dbfs:/datalake/curated/co2_emissions/'

# Uuse repartition() to get one file per partition value
df_co2_emissions = df_co2_emissions.repartition('year')
(
  df_co2_emissions
  .write
  .mode('overwrite')
  .partitionBy('year')
  .option('path',dbfs_base_path)
  .format('parquet')
  .saveAsTable('eea_curated.co2_emissions')
)

Read about parquet format here: https://databricks.com/glossary/what-is-parquet